In [3]:
import os 
from openai import OpenAI

In [4]:
from dotenv import load_dotenv, find_dotenv
load_dotenv()

True

In [10]:
endpoint = "https://models.github.ai/inference"
model_name = "openai/gpt-4o"

client = OpenAI(
    base_url=endpoint,
    api_key=os.environ['GITHUB_API_TOKEN'],
    )

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "What is the capital of England?",
        }
    ],
    temperature=0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name

)



In [11]:
response.choices[0].message.content

'The capital of England is **London**.'

In [44]:
import json

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [45]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [46]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [47]:
# Call the ChatCompletion endpoint
response = client.chat.completions.create(
    # OpenAI Updates: As of June 2024, we are now using the GPT-3.5-Turbo model
    model=model_name,
    messages=messages,
    functions=functions
)

In [48]:
response

ChatCompletion(id='chatcmpl-BYwXDA81YBFhEexIXYenkiZhu6nnI', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather'), tool_calls=None), content_filter_results={})], created=1747667859, model='gpt-4o-2024-11-20', object='chat.completion', service_tier=None, system_fingerprint='fp_ee1d74bde0', usage=CompletionUsage(completion_tokens=16, prompt_tokens=79, total_tokens=95, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered'

In [49]:
response.choices[0].message.function_call

FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather')

In [50]:
response.choices[0].message.function_call.name


'get_current_weather'

In [51]:
args = response.choices[0].message.function_call.arguments

In [52]:
json.loads(args)

{'location': 'Boston'}

In [53]:
args = json.loads(args)

In [54]:
observation = get_current_weather(args)

In [55]:
#function_call param can take : "auto", or "none" or {"name": "get_current_weather"}

In [56]:
messages.append(response.choices[0].message)

In [57]:
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

In [58]:
response = client.chat.completions.create(
    # OpenAI Updates: As of June 2024, we are now using the GPT-3.5-Turbo model
    model=model_name,
    messages=messages
)

In [61]:
response.choices[0].message.content

'The weather in Boston is currently 72°F, sunny, and windy.'